# Sales Forecasting - Model Training & Evaluation
## Comparing Linear Regression and ARIMA Models for Demand Prediction

### 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForest Regressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Time series library
from statsmodels.tsa.arima.model import ARIMA

print("Libraries imported successfully")

### 2. Load and Prepare Data

In [ ]:
# Load the sales data
df = pd.read_csv("data/sales.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
df.head()

### 3. Feature Engineering
Create time-based features for forecasting

In [ ]:
# Lag features
df['lag_1'] = df['sales'].shift(1)
df['lag_3'] = df['sales'].shift(3)

# Rolling statistics
df['rolling_mean_3'] = df['sales'].rolling(window=3).mean()
df['rolling_std_3'] = df['sales'].rolling(window=3).std()

# Calendar features
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day_of_week'] = df['date'].dt.dayofweek

# Remove NaN values
df = df.dropna()

print(f"Features created. New shape: {df.shape}")
print(f"\nFeature columns: {df.drop(['date', 'sales'], axis=1).columns.tolist()}")

### 4. Train-Test Split
Time-based split to maintain temporal order

In [ ]:
# Define split date
split_date = '2023-01-01'

# Split data
train = df[df['date'] < split_date]
test = df[df['date'] >= split_date]

print(f"Train set: {train.shape[0]} samples ({train['date'].min()} to {train['date'].max()})")
print(f"Test set: {test.shape[0]} samples ({test['date'].min()} to {test['date'].max()})")
print(f"Train-test ratio: {len(train)}/{len(test)} ({len(train)/len(df)*100:.1f}% train)")

In [ ]:
# Prepare features and target
feature_cols = ['lag_1', 'lag_3', 'rolling_mean_3', 'rolling_std_3', 'month', 'quarter', 'day_of_week']

X_train = train[feature_cols]
y_train = train['sales']

X_test = test[feature_cols]
y_test = test['sales']

print(f"\nFeature matrix shape: {X_train.shape}")
print(f"Target variable shape: {y_train.shape}")

### 5. Model 1: Linear Regression

In [ ]:
# Train Linear Regression
print("Training Linear Regression model...")
lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions
pred_lr = lr.predict(X_test)

print("Linear Regression training complete")
print(f"Model coefficients: {lr.coef_}")
print(f"Model intercept: {lr.intercept_:.2f}")

### 6. Model 2: Random Forest (Optional Enhancement)

In [ ]:
# Train Random Forest
print("Training Random Forest model...")
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf.fit(X_train, y_train)

# Make predictions
pred_rf = rf.predict(X_test)

print("Random Forest training complete")
print(f"Feature importances: {dict(zip(feature_cols, rf.feature_importances_))}")

### 7. Model 3: ARIMA Time Series Model

In [ ]:
# Train ARIMA model
print("Training ARIMA model...")
print("This may take a moment...")

# ARIMA(p, d, q) - (1,1,1) is a common starting point
arima_model = ARIMA(train['sales'], order=(1, 1, 1))
arima_fit = arima_model.fit()

# Make predictions
pred_arima = arima_fit.forecast(steps=len(test))

print("ARIMA training complete")
print(f"\nARIMA Model Summary:")
print(arima_fit.summary())

### 8. Model Evaluation
Compare models using multiple metrics

In [ ]:
# Calculate metrics for each model
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{model_name} Performance:")
    print(f"  MAE:  {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")
    
    return {'Model': model_name, 'MAE': mae, 'RMSE': rmse, 'R²': r2, 'MAPE': mape}

# Evaluate all models
results = []
results.append(evaluate_model(y_test, pred_lr, "Linear Regression"))
results.append(evaluate_model(y_test, pred_rf, "Random Forest"))
results.append(evaluate_model(y_test, pred_arima, "ARIMA"))

# Create comparison dataframe
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)
results_df

### 9. Visualize Model Predictions

In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(14, 6))

plt.plot(test['date'], y_test.values, label='Actual', linewidth=2, marker='o', markersize=4)
plt.plot(test['date'], pred_lr, label='Linear Regression', linewidth=2, alpha=0.7)
plt.plot(test['date'], pred_rf, label='Random Forest', linewidth=2, alpha=0.7)
plt.plot(test['date'], pred_arima, label='ARIMA', linewidth=2, alpha=0.7)

plt.title("Sales Forecasting: Model Comparison", fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Sales", fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 10. Residual Analysis

In [ ]:
# Calculate residuals
residuals_lr = y_test - pred_lr
residuals_rf = y_test - pred_rf
residuals_arima = y_test - pred_arima

# Plot residuals
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].scatter(pred_lr, residuals_lr, alpha=0.6)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_title('Linear Regression Residuals')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Residuals')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(pred_rf, residuals_rf, alpha=0.6, color='green')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_title('Random Forest Residuals')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Residuals')
axes[1].grid(True, alpha=0.3)

axes[2].scatter(pred_arima, residuals_arima, alpha=0.6, color='orange')
axes[2].axhline(y=0, color='r', linestyle='--')
axes[2].set_title('ARIMA Residuals')
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('Residuals')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 11. Feature Importance (Random Forest)

In [ ]:
# Plot feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 5))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.xlabel('Importance')
plt.title('Feature Importance - Random Forest Model')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 3 Most Important Features:")
print(feature_importance.head(3))

---
## Summary & Insights

### Model Performance Comparison:
Based on the evaluation metrics above:

**Best Model:** [Identify based on lowest MAE/RMSE and highest R²]

### Key Findings:
1. **Linear Regression:** Simple, interpretable baseline model
2. **Random Forest:** Captures non-linear patterns, handles feature interactions
3. **ARIMA:** Specialized for time series, captures temporal dependencies

### Most Important Features:
- Primary driver: [lag_1, rolling_mean_3, etc.]
- Secondary factors: [month, quarter for seasonality]

### Business Recommendations:
1. Use [best model] for production forecasting
2. Focus on [top features] for inventory planning
3. Monitor model performance and retrain quarterly

### Next Steps:
- Hyperparameter tuning for better performance
- Test additional models (XGBoost, Prophet)
- Deploy model for real-time predictions
- Set up monitoring and alerting system